# Bigfoot dashboard example using JupyterDash

This example has been created following the tutorial at https://timothyrenner.github.io/datascience/2017/08/08/finding-bigfoot-with-dash-part-1.html
The dashboard is created via JupyterDash rather than dash, which also allows visualisation within jupyter lab or inline.

In [ ]:
from jupyter_dash import JupyterDash
import pandas as pd
import os
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

#the external stylesheet is needed to be able to have the different columns,
#otherwise it adds everything in rows
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets) 

In [ ]:
#loading the data, from the same folder
BFRO_DATA = pd.read_csv('bfro_report_locations.csv')
BFRO_DATA['timestamp'] = pd.to_datetime(BFRO_DATA['timestamp'])

In [ ]:
#creating a day of the week and a year column for the graphs
year_t = []
day_t = []
for index,row in BFRO_DATA.iterrows():
    year_t.append(row['timestamp'].year)
    day_t.append(row['timestamp'].weekday())
BFRO_DATA['year'] = year_t
BFRO_DATA['day'] = day_t

In [ ]:
#layout and data for the map; the open street map is
#one of the maps that do not require a token
def bigfoot_map(sightings):
    return{'data':[{'type': 'scattermapbox', 
                    'lat': sightings[sightings['classification']==i]['latitude'],
                    'lon': sightings[sightings['classification']==i]['longitude'], 
                    'text': sightings[sightings['classification']==i]['title'],
                    'mode': 'markers', 'name': i, 
                    'marker':{'size': 3, 'opacity': 1.0}
                    } for i in sightings.classification.unique()],
           'layout':{'autosize': True, 'hovermode':'closest', 
                     'mapbox':{'style': 'open-street-map', 
                               'center':{'lat': 40, 'lon': -98.5},
                               'zoom': 2}}}

In [ ]:
#layout and data for the sighting by year chart; it starts with grouping 
#by year and classification and then filtering to exclude outliers in the years                              
def bigfoot_byyear(sightings):
    sightings_byyear = sightings.groupby(['classification','year'])['number'].count()
    sightings_byyear = pd.DataFrame(sightings_byyear).reset_index()
    sightings_byyear = sightings_byyear[(sightings_byyear['year']>=1900) &
                                        (sightings_byyear['year']<=2017)]
    return{'data':[{'type': 'scatter', 'mode': 'lines+markers','name': i, 
                   'x': sightings_byyear[sightings_byyear['classification']==i]['year'],
                   'y': sightings_byyear[sightings_byyear['classification']==i]['number']}
           for i in sightings_byyear.classification.unique()],
           'layout':{'title': 'Sightings by year', 'showlegend': False}}

In [ ]:
#layout and data for the sightings by day of the week
def bigfoot_dow(sightings):
    sightings_bydow = sightings.groupby('day')['number'].count()
    sightings_bydow = pd.DataFrame(sightings_bydow).reset_index()
    sightings_bydow['day2'] = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
    return{'data':[{'type': 'bar', 'x': sightings_bydow['day2'],
                    'y': sightings_bydow['number']}],
           'layout':{'title': 'Sightings by day of the week'}}

In [ ]:
#layout and data for the donut chart with sightings by classification
def bigfoot_class(sightings):
    sightings_class = sightings.groupby('classification')['number'].count()
    sightings_class = pd.DataFrame(sightings_class).reset_index()
    return{'data':[{'type': 'pie', 'labels': sightings_class['classification'],
                    'values': sightings_class['number'], 'hole': 0.4}],
           'layout':{'title': 'Sightings by classification'}}

In [ ]:
#adding the componenets to the layout
#first the overall container, followed by each row, and then columns
#within each row
app.title = 'Bigfoot sightings'
app.layout = html.Div([html.Div([html.Div([html.H1('Bigfoot Sightings',
                                                   className='text-center')],
                                          className='twelve columns')],
                                className='row'),
                       html.Div([html.Div([html.P([html.B('Filter the titles: '),
                                                   dcc.Input(placeholder=
                                                             'Try "heard"',
                                                             id='bigfoot-filter',
                                                             value='')])],
                                          className='six columns'),
                                 html.Div([html.P(['Data pulled from ',
                                                   html.A('bfro.net', 
                                                          href='http://www.bfro.net/')],
                                                  style={'text-align': 'right'})],
                                          className='six columns')],
                                className='row'),
                       html.Div([html.Div([dcc.Graph(id='bigfoot-map')],
                                          className='eight columns'),
                                 html.Div([dcc.Graph(id='bigfoot-dow')],
                                          className='four columns')],
                                className='row'),
                       html.Div([html.Div([dcc.Graph(id='bigfoot-year')],
                                          className='eight columns'),
                                 html.Div([dcc.Graph(id='bigfoot-class')],
                                          className='four columns')],
                                className='row')], 
                      className='container-fluid')

In [ ]:
#adding the various callbacks required for filtering and tailoring the dashboard
@app.callback(Output('bigfoot-map', 'figure'), 
              [Input('bigfoot-filter', 'value')])
def filter_map(filter_text):
    filtered = BFRO_DATA[BFRO_DATA['title'].str.contains(filter_text.lower())]
    return bigfoot_map(filtered)

@app.callback(Output('bigfoot-year', 'figure'),
              [Input('bigfoot-filter', 'value')])
def filter_year(filter_text):
    filtered = BFRO_DATA[BFRO_DATA['title'].str.contains(filter_text.lower())]
    return bigfoot_byyear(filtered)

@app.callback(Output('bigfoot-dow', 'figure'),
              [Input('bigfoot-filter', 'value')])
def filter_dow(filter_text):
    filtered = BFRO_DATA[BFRO_DATA['title'].str.contains(filter_text.lower())]
    return bigfoot_dow(filtered)

@app.callback(Output('bigfoot-class', 'figure'),
              [Input('bigfoot-filter', 'value')])
def filter_class(filter_text):
    filtered = BFRO_DATA[BFRO_DATA['title'].str.contains(filter_text.lower())]
    return bigfoot_class(filtered)

In [ ]:
#by using mode='jupyterlab', it can be run in jupyter lab
app.run_server(mode='external', port = 8090, dev_tools_ui=True, #debug=True,
                  dev_tools_hot_reload =True, threaded=True)